In [ ]:
# Standard and GIS Modules
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import scipy

import warnings
import seaborn as sns
warnings.filterwarnings("ignore")
plt.rcParams.update({"font.size":20})

### Real data

##### Compute LOSH

In [ ]:
from esda.losh import LOSH
import libpysal
# ls = LOSH(connectivity=w, inference="chi-square").fit(boston_ds['NOX'])

dataset_target = {
    "plants": "richness_species_vascular",
    "meuse": "zinc",
    "atlantic": "Rate",
    "deforestation": "deforestation_quantile",
    "california_housing": "median_house_value",
}

dataset_losh = {}
for d in os.listdir("data"):
#     if "meuse" in d: #  or "deforestation" in d:
#         continue
#     print("------------- ", d)
    f = pd.read_csv(os.path.join("data", d))
    coords = f[["x", "y"]]
    
    # with KNN:
    w = libpysal.weights.KNN(coords, k=20)
    
    # extract target var
    target_var = dataset_target[d.split(".")[0]]
    
    ls = LOSH(connectivity=w, inference="chi-square").fit(f[target_var].values)
    
    dataset_losh[(d.split(".")[0]).replace("_", " ")] = np.mean(ls.Hi)
#     f.drop(["x", "y", ]], axis=1, inplace=True)
#     w_cutoff = (np.max(coords, axis=0) - np.min(coords, axis=0)).sum() / 10
#     print(w_cutoff)
#     w = get_weights_as_array(np.array(coords), w_cutoff)
dataset_losh

In [ ]:
# 100
{'plants': 0.9909525898042943,
 'california_housing': 0.9199100144573773,
 'deforestation': 0.9968442529301278,
 'atlantic': 0.9620820900732183}
# 10
{'plants': 1.0165645863370958,
 'california_housing': 0.8750507024465561,
 'deforestation': 0.9969507834341067,
 'atlantic': 1.0031870804023555}
# 20
{'plants': 1.0609760745783343,
 'california_housing': 0.8773202806724744,
 'deforestation': 0.9978335532280674,
 'atlantic': 1.004762023375391}

##### Compute sample size

In [ ]:
sample_dict = {}
feature_dict = {}
for out_file in os.listdir("data"):
    if "folds" in out_file or "synthetic" in out_file or out_file[0] == "." or "deprecated" in out_file:
        continue
    dataset_name = out_file[:-4].replace("_", " ")
    data = pd.read_csv(os.path.join("data", out_file))
    sample_dict[dataset_name] = len(data)
    feature_dict[dataset_name] = data.shape[1] - 2
sample_dict

#### Model recommentation

In [ ]:
recommendation_dict = {"california housing": "RF", "meuse": "GWR", "plants": "GWR / Kriging", "atlantic": "RF", "deforestation": "RF"}

#### Read results

In [ ]:
res_path = "outputs/real_feb_23"

In [ ]:
all_res = []
for out_file in os.listdir(res_path):
    if "folds" in out_file or "synthetic" in out_file or out_file[0] == "." or "deprecated" in out_file:
        continue
    dataset_name = " ".join(out_file.split("_")[1:])[:-4]
    res = pd.read_csv(os.path.join(res_path, out_file))
    res["Dataset"] = dataset_name
    raw_data = pd.read_csv(os.path.join("data", out_file[8:]))
#     res["Samples"] = len(raw_data)
#     res["LOSH"] = round(dataset_losh[out_file[8:-4]], 2)
#     res.sort_values("Method", ascending=False, inplace=True)
    all_res.append(res)
all_res = pd.concat(all_res)

In [ ]:
all_res.loc[all_res["Method"] == "linear regression", "Method"] = "OLS"

In [ ]:
# remove SAR
all_res = all_res[all_res["Method"] != "SAR"]

In [ ]:
# check if all the metrics align in terms of ranking
all(all_res.sort_values(["Dataset", "R-Squared"]).reset_index() == all_res.sort_values(["Dataset", "MAE"]).reset_index())

In [ ]:
pivoted = all_res.pivot(index='Dataset', columns='Method', values='RMSE')
pivoted.loc["california housing"] = pivoted.loc["california housing"].round()
# sorted_columns
pivoted["LOSH"] = pd.Series(dataset_losh)
pivoted["Samples"] = pd.Series(sample_dict)
pivoted["k"] = pd.Series(feature_dict)
pivoted["Recommended model"] = pd.Series(recommendation_dict)
col_order = ['Dataset', "Samples", "k", "LOSH", "Recommended model", 'OLS', 'SLX', 'GWR', 'RF', 'RF (coordinates)', 'spatial RF', 'Kriging']

final = pivoted.reset_index().reset_index(drop=True)[col_order] #.drop_index("Method", axis=1)

In [ ]:
final

In [ ]:
print(final.to_latex(index=False, float_format="%.2f"))

#### Make table

In [ ]:
print(all_res.groupby(["Dataset", "Samples", "LOSH", "Method"]).mean().round(5).to_latex(float_format="%.2f"))

#### Make plot

In [ ]:
# how big is each real dataset:
data_path = "data/"
data_len = {}
for dataset in os.listdir(data_path):
    test = pd.read_csv(data_path+dataset)
#     print(dataset, len(test))
    dataset_name = " ".join(dataset[:-4].split("_"))
    data_len[dataset_name] = f"{dataset_name}\n({len(test)})"
all_res["Dataset"] = all_res["Dataset"].map(data_len)

In [ ]:
plt.rcParams.update({"font.size":20})
plt.figure(figsize=(12,6))
sns.barplot(data=all_res.reset_index(), x="Dataset", y="R-Squared", hue="Method")
# plt.xlabel("Number of samples")
plt.legend(ncol=4, fontsize=15.5)
plt.ylim(0., 1.35)
plt.tight_layout()
plt.xlabel("Dataset", weight="bold")
plt.ylabel("R-Squared score", weight="bold")
plt.savefig("outputs/real_dataset_barplot.pdf")
plt.show()

In [ ]:
results.columns

## Explanatory plots

In [ ]:
weights = np.array([-0.95, 0.38, 0.66, -0.43, 0.22])

nr_data = 10000
nr_feats = 5
feat_cols = ["feat_" + str(i) for i in range(nr_feats)]

coords = np.array([[i, j] for i in range(int(np.sqrt(nr_data))) for j in range(int(np.sqrt(nr_data)))])
coords = coords / np.max(coords) * 2 - 1

spatial_variation = np.zeros((nr_data, nr_feats))
for i in range(nr_feats):
    spatial_variation[:, i] = 0.5 * (
        np.sin(coords[:, 0] * np.pi * 2 + i)
        + np.cos(coords[:, 1] * np.pi * 2 + i)
    )

In [ ]:
cmap = plt.cm.Spectral
fig = plt.figure(figsize=(20,4)) # TODO

########## Number 1
for i in range(5):
    ax = fig.add_subplot(1, 5, i+1)
#     print(weights[i] + spatial_variation[:, i])
    cols = [cmap((val+1) / 2) for val in (weights[i] + 0.5 * spatial_variation[:, i])]
    im = ax.scatter(coords[:, 0], coords[:, 1], c=cols)
    ax.set_xlabel(rf"Coefficient $\beta_{i+1}$")
    ax.set_xticks([])
    ax.set_yticks([])
    ax.spines['bottom'].set_color('white')
    ax.spines['top'].set_color('white') 
    ax.spines['right'].set_color('white')
    ax.spines['left'].set_color('white')

import matplotlib as mpl
cmap = mpl.cm.viridis
bounds = [-1, 2, 5, 7, 12, 15]
cmap = mpl.cm.Spectral
norm = mpl.colors.Normalize(vmin=-1, vmax=1)

fig.subplots_adjust(right=0.88)
cbar_ax = fig.add_axes([0.9, 0.15, 0.02, 0.7])
cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
             cax=cbar_ax)

plt.savefig("outputs/coefficient_figure_5.png")
plt.show()


# ########## Number 2
# for i in range(5):
#     ax = fig.add_subplot(2, 5, i+1 + 5)
# #     print(weights[i] + spatial_variation[:, i])
#     cols = [cmap((val+1) / 2) for val in (weights[i] + 0.4 * spatial_variation[:, i])]
#     im = ax.scatter(coords[:, 0], coords[:, 1], c=cols)
#     ax.set_xlabel("Coefficient "+str(i+1))
#     ax.set_xticks([])
#     ax.set_yticks([])
#     ax.spines['bottom'].set_color('white')
#     ax.spines['top'].set_color('white') 
#     ax.spines['right'].set_color('white')
#     ax.spines['left'].set_color('white')

# import matplotlib as mpl
# cmap = mpl.cm.viridis
# bounds = [-1, 2, 5, 7, 12, 15]
# cmap = mpl.cm.Spectral
# norm = mpl.colors.Normalize(vmin=-1, vmax=1)

# fig.subplots_adjust(right=0.825)
# cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])

# cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
#              cax=cbar_ax)
             
# plt.tight_layout(w_pad=5, h_pad = 5)


### Noise figure

To generate the noise plots, run the above code with the three different types of noise, and execute the plot below with each type

In [ ]:
# CODE FOR ALL THREE NOISE TYPES
noise_level = 0.5

nr_data = 10000
nr_feats = 5
feat_cols = ["feat_" + str(i) for i in range(nr_feats)]

coords = np.array([[i, j] for i in range(int(np.sqrt(nr_data))) for j in range(int(np.sqrt(nr_data)))])
coords = coords / np.max(coords) * 2 - 1
synthetic_data = pd.DataFrame(coords, columns=["x_coord", "y_coord"])

spatial_variation = np.zeros((nr_data, nr_feats))
for i in range(nr_feats):
    spatial_variation[:, i] = 0.5 * (
        np.sin(coords[:, 0] * np.pi * 2 + i)
        + np.cos(coords[:, 1] * np.pi * 2 + i)
    )

    
for noise_type in ["constant", "heterogeneous - same", "heterogeneous - different"]:

    if noise_type == "constant":
        noise = np.random.normal(0, noise_level, nr_data)
    elif noise_type == "heterogeneous - different":
        spatial_variation_different = noise_level * (
            0.5
            * (
                synthetic_data["x_coord"].values
                + synthetic_data["y_coord"].values
            )
            + 1
        )
        noise = np.random.normal(
            0,
            spatial_variation_different,
            len(spatial_variation_different),
        )
    elif noise_type == "heterogeneous - same":
        # e.g. high noise level (0.5), spatial variation is from
        # sin and cos so it's between -1 and 1, so we make + 1
        # so on average we multiply by 1, but varying variance
        # between 0.5 * 0 and 0.5 * 2
        spatially_dependent_noise = noise_level * (
            spatial_variation[:, 0] + 1  # without locality level!
        )
        noise = np.random.normal(
            0, spatially_dependent_noise, nr_data
        )
    else:
        raise RuntimeError("Noise must be one of above")

    plt.figure(figsize=(6,4))
    plt.scatter(coords[:, 0], coords[:, 1], c=noise, vmin=-3, vmax=3)
    plt.colorbar()
    # plt.title(f"Distribution $\epsilon$ ({noise_type})", fontsize=15)
    plt.axis("off")
    plt.tight_layout()
    plt.savefig(f"outputs/noise_{noise_type}.png")
    plt.show()

# Synthetic experiment - load results

In [ ]:
def add_discrete_noise(results):
    results["noise_discrete"] = pd.NA
    results["locality_discrete"] = pd.NA
    results.loc[results["noise"] < 0.3,  "noise_discrete"] = "low"
    results.loc[results["noise"] >= 0.3,  "noise_discrete"] = "high"
    results.loc[results["locality"] < 0.3,  "locality_discrete"] = "low"
    results.loc[results["locality"] >= 0.3,  "locality_discrete"] = "high"
    return results

In [ ]:
path = "outputs/syn_feb_23"

In [ ]:
# # merge the three files
# results = []
# for noise in ["uniformly_distributed", "heterogeneous_-_same", "heterogeneous_-_different"]:
#     results.append(pd.read_csv(os.path.join(path, "synthetic_data_results_"+ noise+".csv")))
# results = pd.concat(results)
# results.loc[results["model"] == "linear regression", "model"] = "OLS"
# results = add_discrete_noise(results)

In [ ]:
noise_level_range = [0, 0.1, 0.2, 0.3, 0.4, 0.5]
locality_range = [0, 0.1, 0.2, 0.3, 0.4, 0.5]

In [ ]:
use_function = "non-linear 2"

In [ ]:
# take files from the testing-directory and merge them with the linear results
non_linear_dir = "new_non_linear"
test_dir = "multiple_nonlinear_tests"

# merge the three files
results = []
for noise in ["uniformly_distributed", "heterogeneous_-_same", "heterogeneous_-_different", "train"]:
    results2 = pd.read_csv(os.path.join(path, non_linear_dir, test_dir, "synthetic_data_results_"+ noise+".csv"))
    results1 = pd.read_csv(os.path.join(path, "synthetic_data_results_"+ noise+".csv"))
    # only use non-linear results from file 1 and linear ones from file 2
    results2 = results2[results2["data mode"] == use_function]
    results1 = results1[results1["data mode"] == "linear"]
    # concat
    results_one = pd.concat((results1, results2))
    # post proressing
    results_one.loc[results_one["model"] == "linear regression", "model"] = "OLS"
    results_one.loc[results_one["data mode"] == use_function, "data mode"] = "non-linear"
    results_one.to_csv(os.path.join(path, non_linear_dir, "synthetic_data_results_"+ noise+".csv"))
    if noise != "train":
        results.append(results_one)

results = pd.concat(results)
# general preprocesing steps
results = add_discrete_noise(results)
print(len(results)//2)

### Main plot

In [ ]:
include_models = ['OLS', 'SLX', 'GWR', 'RF', 'RF (coordinates)', 'spatial RF', 'Kriging']


In [ ]:
def main_plot(results, nr_data=500, noise_type="uniformly distributed", save_path="outputs/main_plot.pdf", score_col="RMSE"):
    include_models = [m for m in include_models if m in results["model"].unique()]
    include_function = ["linear", "non-linear"]
    # [model for model in results["model"].unique() if "geo" not in model]
    nr_models = len(include_models)
    fig = plt.figure(figsize=(15, 4))
    #fig = plt.figure(figsize=(16, 6.5))
    for mode_ind, mode in enumerate(include_function):
    #     print("----------------")
        for model_ind, model in enumerate(include_models):
    #         print(mode, "data, --> model:", model)
            results_filter = results[
                (results["data mode"] == mode) & 
                (results["model"] == model) & 
                (results["nr_data"] == nr_data) & 
                (results["noise_type"] == noise_type)
            ]
            results_filter.set_index(["noise", "locality"], inplace=True)
            visualize_scores = np.zeros((len(noise_level_range), len(locality_range)))
            for i, noise in enumerate(noise_level_range):
                for j, locality in enumerate(locality_range):
                    score = results_filter.loc[noise, locality][score_col].mean()
                    visualize_scores[i, j] = score

            ax1 = fig.add_subplot(len(include_function), nr_models+1, ((nr_models+1) * mode_ind) + model_ind+1)
            imshow_plot = ax1.imshow(visualize_scores, vmin=0, vmax=0.9)
    #         plt.axis("off")
#             if model_ind==0:
#                 ax1.set_ylabel("$\longleftarrow$ Increasing \n noise", fontsize=15)
#             ax1.yaxis.set_label_position("right")
#             ax1.yaxis.tick_right()
            plt.xticks([])
            plt.yticks([])
#             ax1.set_xlabel("$\longrightarrow$ decreasing \n stationarity", fontsize=10)
            if model_ind == 0:
    #             ax2 = ax1.twinx()
    #             ax2.set_ylabel(mode)
    #             ax2.yaxis.set_label_position("right")
#                 pad = 2
                mode_new = "non-linear\n(simple)  " if mode == "non-linear (simple)" else mode
                ax1.annotate(mode_new, xy=(0, 0.5), xytext=(-50, 0), # ax1.yaxis.labelpad - pad
                    xycoords=ax1.yaxis.label, textcoords='offset points',
                    size=18, ha='right', va='center', rotation=90, weight="bold")
            if mode_ind == 0:
                ax1.set_title(model, weight="bold", fontsize=15)
    
    fig.text(0.5, 0.0, "$\longrightarrow$ decreasing stationarity", ha='center')
#     fig.text(0.5, 0.36, "$\longrightarrow$ decreasing stationarity", ha='center')
#     fig.text(0.5, 0.7, "$\longrightarrow$ decreasing stationarity", ha='center')
    
    fig.text(0.06, 0.45, "$\longleftarrow$ Increasing noise", va='center', rotation='vertical')
    # make colorbar
    # fig.subplots_adjust(right=0.95)
    cbar_ax = fig.add_axes([0.88, 0.05, 0.02, 0.9])
    fig.colorbar(imshow_plot, cax=cbar_ax, label=score_col)
    plt.tight_layout()
    if save_path is not None:
        plt.savefig(save_path)
    plt.show()

In [ ]:
main_plot(results, nr_data=1000, save_path ="outputs/main_plot.pdf")

### Barplot - low noise, 0.3 non-stationarity, over samples


In [ ]:
score_col = "RMSE"

In [ ]:
# only look at local models
subset = results[
    (results["model"] != "SAR")
     # .isin(["GWR", "RF", "RF (coordinates)", "spatial RF", "Kriging"])) # "spatial RF",
]

In [ ]:
plt.rcParams.update({"font.size":22})

In [ ]:
# subset.groupby(["nr_data", "data mode", "model", "noise_discrete", "locality_discrete"]).agg({"R2 score": "mean"})
plt.figure(figsize=(18,6))
counter = 1
modes = ["linear", "non-linear"]
for mode, save_name in zip(modes, ["linear", "non_linear"]):
    plt.subplot(1, len(modes), counter)
    counter += 1
    subset_2 = subset[
        (subset["data mode"] == mode) &
        (subset["noise_discrete"] == "low") & 
        (subset["locality_discrete"] == "high") & 
#         (subset["noise"] == 0.1) & (subset["locality"] == 0.4)&
        (subset["noise_type"] == "uniformly distributed")
    ]
    subset_2 = subset_2.groupby(["nr_data", "model"]).agg({score_col: "mean"})

    ax = sns.barplot(data=subset_2.reset_index().set_index("model").loc[include_models].reset_index(), x="nr_data", y=score_col, hue="model")
    plt.ylim(0, 0.7)
    plt.xlabel("Number of samples")
#     if mode == "non-linear (simple)":
#         plt.legend(title="Model", loc="lower right", framealpha=1, ncol=2)
#     else:
    plt.legend([],[], frameon=False)
    plt.title(mode+" DGP")
    
handles, labels = ax.get_legend_handles_labels()
plt.tight_layout()
plt.figlegend(handles, labels, loc = 'upper center', ncol=7, labelspacing=0., bbox_to_anchor=(0.5,1.09))
plt.savefig(f"outputs/barplot_main.pdf", bbox_inches="tight")
plt.show()
    

### Noise type analysis

In [ ]:
fontsize=18
plt.rcParams.update({"font.size":fontsize})

In [ ]:
subset = results[
    results["model"].isin(["OLS", "GWR", "RF (coordinates)", "Kriging"]) # "spatial RF",
]
subset["noise_type"] = subset["noise_type"].map({
    'uniformly distributed':'uniformly distributed noise', 'heterogeneous - same': 'heterogeneous (trigonometric)',
       'heterogeneous - different': "heterogeneous (linear)" 
})

In [ ]:
# fig = plt.figure(figsize=(12, 9))
fig = plt.figure(figsize=(13, 4))
counter = 1
for i, mode in enumerate(["linear", "non-linear"]): #  "non-linear (simple)" # linear", 
    for j, model in enumerate(["GWR", "Kriging"]):
        subset2 = subset[
                (subset["model"] == model) &
                (subset["data mode"] == mode) &
        #         (subset["noise_discrete"] == "low") & 
                (subset["locality_discrete"] == "high") & 
        #         (subset["noise"] == 0.3) & 
#                 (subset["locality"] == 0.4) & 
        #         (subset["noise_type"] == "constant") &
                (subset["nr_data"] == 500)
        ]
#         subset2["noise_type"] = subset2["noise_type"] + " noise"
        ax = fig.add_subplot(1, 4, counter)
        sns.lineplot(ax=ax, data=subset2.reset_index(), x ="noise", y="RMSE", hue="noise_type")
#         if counter == 1:
#             plt.legend(title="Noise (spatial distribution)")# , loc=(1, 1))
#         else:
        plt.legend([], [], frameon=False)
        plt.xlabel(r"Noise level $\sigma$")
    
        plt.title(f"{model}\n{mode} DGP", fontsize=fontsize)
        if counter > 1:
            plt.ylabel("")
            plt.yticks([], [])
#         if j == 0:
#             ax.annotate(mode, xy=(0, 0.5), xytext=(-20, 0), # ax1.yaxis.labelpad - pad
#                     xycoords=ax.yaxis.label, textcoords='offset points',
#                     size='large', ha='right', va='center', rotation=90, weight="bold")
#         ax.set_xlabel("Noise level ($\sigma$)")
        ax.set_ylim(0, 0.8)
#         if counter in [2, 4]:
#             plt.ylabel("")
#         if counter in [1, 2]:
#             plt.title(model, weight="bold", fontsize=16)
        counter += 1

handles, labels = ax.get_legend_handles_labels()
plt.tight_layout()
plt.figlegend(handles, labels, loc = 'upper center', ncol=3, labelspacing=0., bbox_to_anchor=(0.5,1.09))
plt.savefig("outputs/noise_analysis.pdf", bbox_inches="tight")
plt.show()

### Runtime

In [ ]:
time_plot_data = results.groupby(["model", "nr_data"])["time"].mean()
time_plot_data = time_plot_data.loc[['OLS', 'SLX', 'GWR', 'RF', 'RF (coordinates)', 'spatial RF', 'Kriging']]

In [ ]:
plt.figure(figsize=(10,4))
sns.barplot(data=time_plot_data.reset_index(), x="nr_data", y="time", hue="model")
plt.yscale("log")
plt.ylabel("Runtime (seconds)")
plt.xlabel("Number of samples")
plt.legend(ncol=3, loc="upper left")
plt.ylim(0, 800)
plt.tight_layout()
plt.savefig("outputs/runtime_plot.pdf")
plt.show()

### Train vs test score

In [ ]:
train_results = pd.read_csv(os.path.join(path, 'new_non_linear', "synthetic_data_results_train.csv"))
train_results.loc[train_results["model"] == "linear regression", "model"] = "OLS"
test_results = results[results["noise_type"] == "uniformly distributed"]# .drop("Unnamed: 0", axis=1)
train_results["evaluation"] = "training error"
test_results["evaluation"] = "testing error"
print(len(train_results), len(test_results))
traintest = pd.concat((train_results, test_results))
# traintest["R2 score"] = traintest["R2 score"].clip(-5, 1)

In [ ]:
fig = plt.figure(figsize=(15, 4)) # 10
subset = traintest[
    (traintest["model"].isin(["SLX", "GWR"])) # & # "spatial RF",
    & (traintest["nr_data"] == 1000)
].sort_values("model", ascending=False)
counter = 1
modes_considered = ["linear"] # ["linear", "non-linear"]
signal_to_noise_modes = [
        "Weak non-stationarity ($\lambda$) \n and weak noise ($\sigma$)",
        "Strong non-stationarity ($\lambda$) \n and weak noise ($\sigma$)",
        "Weak non-stationarity ($\lambda$) \n and strong noise ($\sigma$)",
        "Strong non-stationarity ($\lambda$) \n and strong noise ($\sigma$)",
]
for i, mode in enumerate(modes_considered): #  "non-linear (simple)" # linear", 
    for j, greatersmaller in enumerate(signal_to_noise_modes):
        #"Low $\lambda$ and low $\sigma$",
         #                               "High $\lambda$ and low $\sigma$", 
          #                              "Low $\lambda$ and high $\sigma$"]):
        if greatersmaller == "Weak non-stationarity ($\lambda$) \n and weak noise ($\sigma$)":
            subset2 = subset[
                (subset["noise"] < 0.4) & 
                (subset["locality"] < 0.4)
            ]
        elif greatersmaller == "Strong non-stationarity ($\lambda$) \n and weak noise ($\sigma$)":
            subset2 = subset[
                (subset["noise"] < 0.4) & 
                (subset["locality"] >= 0.4)
            ]
        elif greatersmaller == "Strong non-stationarity ($\lambda$) \n and strong noise ($\sigma$)":
            subset2 = subset[
                (subset["noise"] >= 0.4) & 
                (subset["locality"] >= 0.4)
            ]
        elif greatersmaller == "Weak non-stationarity ($\lambda$) \n and strong noise ($\sigma$)":
            subset2 = subset[
                (subset["noise"] >= 0.4) & 
                (subset["locality"] < 0.4)
            ]
        subset2 = subset2[subset2["data mode"] == mode]
        
        if i==0 and j==1: 
            print(subset2.groupby(["model", "evaluation"]).agg({"RMSE": "mean"}))
        ax = fig.add_subplot(len(modes_considered), len(signal_to_noise_modes), counter)
        sns.barplot(ax=ax, data=subset2, x="model", y="RMSE", hue="evaluation")
#         if counter == 2:
#             plt.legend(title="Evaluation data", ncol=1, framealpha=1, loc="lower center")
#         else:
        ymax = 0.59
        plt.legend([], [], frameon=False)
        if j == 0:
            ax.annotate(mode, xy=(0, 0.5), xytext=(-20, 0), # ax1.yaxis.labelpad - pad
                    xycoords=ax.yaxis.label, textcoords='offset points',
                    size=19, ha='right', va='center', rotation=90, weight="bold")
        if i == len(modes_considered)-2:
            plt.xticks([], [])
        if j > 0:
            plt.yticks([], [])
        else:
            plt.yticks(np.arange(0, ymax, 0.1), np.around(np.arange(0, ymax, 0.1), 1))
#         ax.set_xlabel("Noise level ($\sigma$)")
        ax.set_ylim(0, ymax)
        ax.set_xlabel("")
        if counter in [2, 3, 4]:
            plt.ylabel("")
        if counter <= len(signal_to_noise_modes):
            plt.title(greatersmaller, weight="bold", fontsize=15)
        counter += 1
plt.tight_layout()

handles, labels = ax.get_legend_handles_labels()
plt.tight_layout()
plt.figlegend(handles, labels, loc = 'upper center', ncol=2, labelspacing=0., bbox_to_anchor=(0.5,1.1))
plt.savefig("outputs/train_analysis.pdf", bbox_inches="tight")
plt.show()

In [ ]:
 0.198287 / 0.153864, 0.22/0.189239, 0.195645 / 0.149066

### Geo rf vs spatial rf (with old results)

In [ ]:
res_old = pd.read_csv("outputs/synthetic_results_nov_22/synthetic_results.csv")
res_old = res_old[res_old["model"].isin(['geographical RF', 'spatial RF']) & (res_old["nr_data"] < 1000)]

In [ ]:
spatial_rf_better = res_old.set_index("model")
print("Spatial RF has better R2 score in percent cases:",
    sum(spatial_rf_better.loc["spatial RF"]["R2 score"].values >= spatial_rf_better.loc["geographical RF"]["R2 score"].values) / (len(spatial_rf_better) / 2)
     )

In [ ]:
# # First try: model on x axis and function mode on y axis
# plt.figure(figsize=(10, 4))
# res_old.loc[res_old["R2 score"] < 0, "R2 score"] = 0
# sns.barplot(data=res_old, x="model", y="R2 score", hue="data mode")
# plt.legend(ncol=3, fontsize=17, loc="upper center")
# plt.ylim(0, 1)

plt.rcParams.update({"font.size":18})
plt.figure(figsize=(7, 4.4))
res_old.loc[res_old["R2 score"] < 0, "R2 score"] = 0
sns.barplot(data=res_old, x="data mode", y="R2 score", hue="model")
plt.legend(ncol=2, fontsize=18, loc="upper center") # , title="Model")
plt.xlabel("Function mode")
plt.ylim(0, 1)
plt.tight_layout()
plt.savefig("outputs/geo_vs_spatial_rf.pdf")
plt.show()

## GWR coefficient analysis

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import scipy
import warnings

from sklearn.metrics import r2_score
from mgwr.gwr import GWR
from mgwr.sel_bw import Sel_BW

np.random.seed(42)



def non_linear_function_simple(feat_arr, weights):
    if len(weights.shape) == 1:
        weights = np.expand_dims(weights, 0)
    function_zoo = [
        np.sin,
        np.exp,
        lambda x: x ** 2,
        lambda x: x,
        np.cos,
        lambda x: np.log(x ** 2),
    ]
    feature_transformed = np.zeros(feat_arr.shape)
    for i in range(feat_arr.shape[1]):
        feature_transformed[:, i] = (
            function_zoo[i](feat_arr[:, i]) * weights[:, i]
        )
    return np.sum(feature_transformed, axis=1)

nr_feats = 5
max_depth = 30
noise_type = "constant"

locality = 0.3

weights = np.array([-0.95, 0.38, 0.66, -0.43, 0.22])

nr_data = 1000

# MAKE MAIN DATA
train_cutoff = int(nr_data * 0.9)
feat_cols = ["feat_" + str(i) for i in range(nr_feats)]
synthetic_data = pd.DataFrame(
    np.random.rand(nr_data, 2 + nr_feats) * 2 - 1,
    columns=["x_coord", "y_coord"] + feat_cols,
)

# simulate spatial variation of features (varying per weight)
spatial_variation = np.zeros((nr_data, nr_feats))
for i in range(nr_feats):
    spatial_variation[:, i] = 0.5 * (
        np.sin(synthetic_data["x_coord"].values * np.pi * 2 + i)
        + np.cos(synthetic_data["y_coord"].values * np.pi * 2 + i)
    )
                
spatially_dependent_weights = weights + locality * spatial_variation

synthetic_data["label"] = non_linear_function_simple(
                        synthetic_data[feat_cols].values,
                        spatially_dependent_weights,
                    )

param_arr = [spatially_dependent_weights[:train_cutoff, 0]]
for noise_level in [0, 0.3, 0.5]:
    noise = np.random.normal(0, noise_level, nr_data)
    synthetic_data["label"] = synthetic_data["label"] + noise

    train_data, test_data = (
                        synthetic_data[:train_cutoff],
                        synthetic_data[train_cutoff:],
                    )

    train_coords = np.array(train_data[["x_coord", "y_coord"]])
    train_y = np.expand_dims(train_data["label"].values, 1)
    train_x = np.array(train_data[feat_cols])
    # bandwidth selection
    # import pickle
    gwr_selector = Sel_BW(
        train_coords, train_y, train_x, fixed=True, kernel="exponential"
    )
    gwr_bw = gwr_selector.search(criterion="AICc")
    # create and train model
    gwr_model = GWR(
        train_coords,
        train_y,
        train_x,
        gwr_bw,
        kernel="exponential",
        fixed=True,
    )
    gwr_results = gwr_model.fit()

    test_coords = np.array(test_data[["x_coord", "y_coord"]])
    test_x = np.array(test_data[feat_cols])
    # predict
    test_pred = gwr_model.predict(
        test_coords, test_x, gwr_results.scale, gwr_results.resid_response
    ).predictions


    score = r2_score(test_pred, test_data["label"])
    print(score)
    param_arr.append(gwr_results.params[:, 1])

In [ ]:
plt.figure(figsize=(16,4))
names = [r"Real $\beta_1$", r"GWR $\beta_1$ ($\sigma=0$)", r"GWR $\beta_1$ ($\sigma=0.3$)",r"GWR $\beta_1$ ($\sigma=0.5$)"]
for i in range(4):
    plt.subplot(1,4,i+1)
    plt.scatter(train_data["x_coord"], train_data["y_coord"], c=param_arr[i])
    plt.title(names[i], fontsize=18)
    plt.axis("off")
plt.tight_layout()
plt.savefig("outputs/gwr_beta_comparison.png")
plt.show()